In [1]:
import sys
from pathlib import Path

# Add project root to Python path for src imports
PROJECT_ROOT = Path("..").resolve()
sys.path.append(str(PROJECT_ROOT))

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Custom functions
from src.models import get_random_forest
from src.evaluation import (
    compute_roc_auc,
    plot_roc_curve,
    compute_classification_metrics,
    plot_confusion_matrix
)

# Directories
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"
RESULTS_DIR = PROJECT_ROOT / "results"
RESULTS_FIGURES = RESULTS_DIR / "figures"
RESULTS_METRICS = RESULTS_DIR / "metrics"

# Create folders if missing
RESULTS_FIGURES.mkdir(parents=True, exist_ok=True)
RESULTS_METRICS.mkdir(parents=True, exist_ok=True)


In [2]:
# Load descriptor features
X_desc = pd.read_csv(DATA_PROCESSED / "tox21_descriptors.csv")

# Drop non-feature columns if they exist
X_desc = X_desc.drop(columns=["smiles", "mol_id"], errors="ignore")

# Replace infinite values with NaN
X_desc = X_desc.replace([np.inf, -np.inf], np.nan)

print("Initial descriptor shape:", X_desc.shape)
print("Total NaNs in descriptors:", X_desc.isna().sum().sum())

# Fill remaining NaNs with median (robust for chemical descriptors)
X_desc = X_desc.fillna(X_desc.median())

# Convert to NumPy array for modeling
X = X_desc.values

print("Final X shape:", X.shape)
print("Any NaNs left in X?", np.isnan(X).any())


Initial descriptor shape: (3074, 217)
Total NaNs in descriptors: 442
Final X shape: (3074, 217)
Any NaNs left in X? False


In [3]:
# Load cleaned assay labels
tox21 = pd.read_csv(DATA_PROCESSED / "tox21_clean.csv")

# Assay columns
ASSAY_COLUMNS = [
    'NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase',
    'NR-ER', 'NR-ER-LBD', 'NR-PPAR-gamma',
    'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53'
]

y = tox21[ASSAY_COLUMNS]

print("y shape (assays):", y.shape)


y shape (assays): (3074, 12)


In [4]:
sys.path.append(str(PROJECT_ROOT))


In [5]:
from sklearn.metrics import f1_score

metrics_list = []

for target in y.columns:
    print(f"\nEvaluating model for {target}")

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y[target],
        test_size=0.2,
        stratify=y[target],
        random_state=42
    )

    # Initialize class-weighted RandomForest
    model = get_random_forest(class_weight="balanced")
    model.fit(X_train, y_train)

    # Predict probabilities
    y_pred_prob = model.predict_proba(X_test)[:, 1]

    # ROC-AUC
    roc_auc = compute_roc_auc(y_test, y_pred_prob)

    # Optimize threshold based on F1-score
    thresholds = np.linspace(0, 1, 101)
    f1_scores = [f1_score(y_test, (y_pred_prob >= t).astype(int), zero_division=0) for t in thresholds]
    best_idx = np.argmax(f1_scores)
    best_threshold = thresholds[best_idx]
    y_pred = (y_pred_prob >= best_threshold).astype(int)

    # Compute metrics
    precision, recall, f1 = compute_classification_metrics(y_test, y_pred)

    # Plot ROC curve
    plot_roc_curve(
        y_test,
        y_pred_prob,
        title=f"ROC Curve — {target}",
        save_path=RESULTS_FIGURES / f"roc_{target}.png"
    )

    # Plot confusion matrix
    plot_confusion_matrix(
        y_test,
        y_pred,
        title=f"Confusion Matrix — {target}",
        save_path=RESULTS_FIGURES / f"cm_{target}.png"
    )

    # Save results
    metrics_list.append({
        "assay": target,
        "roc_auc": roc_auc,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "best_threshold": best_threshold
    })



Evaluating model for NR-AR

Evaluating model for NR-AR-LBD

Evaluating model for NR-AhR

Evaluating model for NR-Aromatase

Evaluating model for NR-ER

Evaluating model for NR-ER-LBD

Evaluating model for NR-PPAR-gamma

Evaluating model for SR-ARE

Evaluating model for SR-ATAD5

Evaluating model for SR-HSE

Evaluating model for SR-MMP

Evaluating model for SR-p53


In [6]:
# Convert metrics list to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Save to CSV
metrics_df.to_csv(RESULTS_METRICS / "model_metrics.csv", index=False)

# Display final metrics
metrics_df


,assay,roc_auc,precision,recall,f1_score,best_threshold
0,NR-AR,0.665077,0.400000,0.500000,0.444444,0.12
1,NR-AR-LBD,0.642505,0.666667,0.285714,0.400000,0.23
2,NR-AhR,0.844703,0.263158,0.483871,0.340909,0.14
3,NR-Aromatase,0.756171,0.200000,0.181818,0.190476,0.15
4,NR-ER,0.637566,0.276596,0.270833,0.273684,0.17
5,NR-ER-LBD,0.528750,0.400000,0.153846,0.222222,0.30
6,NR-PPAR-gamma,0.884918,0.500000,0.200000,0.285714,0.17
7,SR-ARE,0.707131,0.342857,0.307692,0.324324,0.16
8,SR-ATAD5,0.557096,0.003252,1.000000,0.006483,0.00
9,SR-HSE,0.632397,0.285714,0.200000,0.235294,0.13
